# Этап 11: Mixed-Precision Quantization (Смешанная точность)

Не все слои в нейросети одинаково важны для финального результата. Некоторые можно сжать до 4 бит без потерь, а другие требуют 16 бит для стабильности.

### Идея метода:
Мы используем высокую точность для чувствительных слоев (обычно это эмбеддинги и финальный слой) и низкую — для промежуточных MLP, которые занимают 80% весов.

In [ ]:
import torch
import torch.nn as nn
from src.model import GPTLanguageModel, device

model = GPTLanguageModel().to(device)
model.load_state_dict(torch.load('model_ckpt.pt', map_location=device))

def apply_mixed_precision(mdl):
    print(f"{'Layer':<40} | {'Policy'}")
    print("-" * 55)
    
    for name, param in mdl.named_parameters():
        if 'token_embedding' in name or 'lm_head' in name:
            # Эмбеддинги и голову оставляем в FP16/FP32
            policy = "KEEP FP32"
        elif 'ffwd' in name and 'weight' in name:
            # MLP слои сжимаем агрессивно
            policy = "COMPRESS INT4"
        else:
            policy = "COMPRESS INT8"
            
        print(f"{name[:40]:<40} | {policy}")

apply_mixed_precision(model)
print("\nТакой подход позволяет сохранить 99.9% точности при сжатии модели в 3-4 раза.")